# REDUCCION DE DIMENSIONALIDAD

Muchos de los problemas de Machine Learning implican miles e incluso millones de características por cada instancia de entrenamiento. Esto no solo hace que el entrenamiento sea extremadamente lento si no que también puede hacer mucho más difícil encontrar una buena solución, como veremos. A menudo el problema es referido como la *maldición de la dimensionalidad*.

Afortunadamente, en los problemas del mundo real, a menudo es posible reducir considerablemente el número de características, convirtiendo un problema intratable en uno manejable. Por ejemplo, consideramos las imágenes MNIST (que vimos en el capítulo 3): los píxeles de los bordes de la imagen son casi siempre blancos, por lo que podríamos eliminarlos completamente del conjunto de entrenamiento sin perder mucha información. La siguiente figura ya nos confirmó que dichos píxeles no son importantes en absoluto para la tarea de clasificación. Además, dos píxeles vecinos a menudo están altamente correlacionados: si los combinamos en un único píxel (por ejemplo, tomando la media de las intensidades de los dos píxeles) no perderemos mucha información.

![mnist_pixel_importance](images/ch07/mnist_pixel_importance.png)

---

La reducción de la dimensionalidad pierde algo de información (al igual que comprimir una imagen a JPEG puede degradar su calidad), por lo que, aunque acelerará el entrenamiento, también puede hacer que nuestro sistema se ejecute peor. También hace nuestros pipelines un poco más complejos y, por tanto, difíciles de mantener. Por lo tanto, primero deberíamos intentar entrenar nuestro sistema con los datos originales antes de considerar el uso de la reducción de la dimensionalidad si el entrenamiento es demasiado lento. En algunos casos, sin embargo, reducir la dimensionalidad de los datos de entrenamiento pueden filtrar algún ruido y detalles innecesarios y, por tanto, resultar en una mayor ejecución (pero en general no es así, solo acelerará el entrenamiento).

---

Además de acelerar el entrenamiento, la reducción de dimensionalidad también es extremadamente útil para la visualización de datos (o *DataViz*). Reducir el número de dimensiones a dos (o tres) hace posible elaborar en un gráfico una vista condensada de un conjunto de entrenamiento de gran dimensionalidad y, a menudo, conseguir ideas importantes detectando patrones visualmente, tales como clusters. Además, DataViz es esencial para comunicar nuestras conclusiones a personas que no son científicos de datos, en particular responsables de tomar decisiones que usarán nuestros resultados.

En este capítulo discutiremos la maldición de la dimensionalidad y tendremos una idea de lo que sucede en el espacio de alta dimensionalidad. Después, presentaremos los dos principales enfoques de la reducción de dimensionalidad (proyección y Manifold Learning) e iremos a través de las tres técnicas más populares de reducción de dimensionalidad: PCA, Kernel PCA y LLE.

# La maldición de la dimensionalidad

Estamos tan acostumbrados a vivir en tres dimensiones (bueno, cuatro dimensiones si contamos el tiempo y algunas más si eres un teórico de cuerdas), que nuestra intuición nos falla cuando intentamos imaginar un espacio de alta dimensionalidad. Incluso un básico hipercubo 4D es increiblemente difícil de dibujar en nuestra mente (veamos la siguiente figura), mucho menos un elipsoide 200D doblado en un espacion 100D.

***Punto, segmento, cuadrado, cubo y [tesseract](https://homl.info/30) (hipercubos 0D a 4D)***

![hypercube](images/ch08/hypercube.png)

Resulta que muchas cosas se comportan muy diferentemente en un espacio de alta dimensionalidad. Por ejemplo, si seleccionamos un punto aleatorio en un cuadrado unitario (un cuadrado 1 x 1), tendremos solo una probabilidad de que esté localizado a 0.001 de un borde (en otras palabras, es muy improbable que un punto aleatorio sea "extremo" a lo largo de cualquier dimensión). Pero en un hipercubo unitario 10.000D (un cubo de 1 x 1 x ... 1, con diezmil unos), esta probabilidad es mayor que 99,999999%. La mayoría de los puntos en un hipercubo de alta dimensión están muy cerca del borde. *Dato curioso: cualquier persona que conozca es probablemente extremista en al menos una dimensión (por ejemplo, la cantidad de azúcar que ponen en su café), si se consideran suficientes dimensiones.*

Aquí hay una diferencia más problemática: si seleccionamos aleatoriamente dos puntos en un cuadrado unitario, la distancia entre ambos puntos será, de media, alrededor de 0,52. Si seleccionamos aleatoriamente dos puntos en un cubo unitario 3D, la distancia media será aproximadamente 0,66. Pero ¿qué pasa si seleccionamos aleatoriamente dos puntos en un hipercubo 1.000.000D? Bueno, lo creas o no, será alrededor de 408,25 (aproximadamente $\sqrt{\frac{1.000.000}{6}}$). Esto es bastante contraintuitivo: ¿cómo pueden estar dos puntos tan separados cuando están dentro del mismo hipercubo unitario? Este hecho implica que los datasets de alta dimensionalidad corren el riesgo de eser muy dispersos: la mayoría de las instancias de entrenamiento estarán probablemente muy lejos las unas de las otras. Por supuesto, esto también significa que una nueva instancia estará probablemente lejos de cualquier instancia de entrenamiento, lo que hace que las predicciones sean mucho menos confiables que en menores dimensiones, ya que se basarán en extrapolaciones mucho más grandes. En resumen, cuantas más dimensiones tenga el conjunto de entrenamiento, mayor será el riesgo de sobreajustarlo.

En teoría, una solución a la maldición de la dimensionalidad podría ser incrementar el tamaño del conjunto de entrenamiento para alcanzar una densidad suficiente de instancias de entrenamiento. Desafortunadamente, en la práctica el número de instancias requeridas para alcanzar una densidad dada crece exponencialmente con el número de dimensiones. Con solo 100 características (muchas menos que el problema MNIST), necesitaríamos más instancias de entrenamiento que átomos en el universo observable para que las instancias de entrenamiento estuvieran a una distancia unas de otras de 0,1 de media, asumiendo que se extendieran uniformemente a través de todas las dimensiones.

# Principales enfoques de la Reducción de Dimensionalidad